#ПЭК


скрипты сбора данных для конкурентного анализа цен, применяемых в тарифообразовании

## Методы и построение запроса

документация по интеграции, список методов:
https://kabinet.pecom.ru/api/v1

метод список всех филиалов и городов: https://kabinet.pecom.ru/api/v1/branches/all/ для создания справочников, очищенные данные: основные терминалы/ дополнительные терминалы ниже

построение запроса для расчета стоимости

In [ ]:
{
   "currencyCode": "643",# Код валюты [String]. Необязательно. По умолчанию 643 - российский рубль. Допустимые валюты см.в ответе метода /currency/all/
   "types": [ 1 ],# ОБЯЗАТЕЛЬНО!!! Массив идентификаторов продуктов/тарифов ПЭК для расчета [Array][Number]. Обязательно. Полный список доступных в API продуктов/тарифов можно получить методом /typesOfDelivery/all/. ВНИМАНИЕ! В настоящее время метод не позволяет рассчитывать стоимость по продукту/тарифу 5 - ПЭК:Express Авто
   "senderWarehouseId": "dc6c746d-812d-11e4-bbfc-001999d8b3c5",# ОБЯЗАТЕЛЬНО!!! Идентификатор склада отправителя [String]
   "receiverWarehouseId": "b436c978-086d-11e6-b6ca-00155d668909",# ОБЯЗАТЕЛЬНО!!!Идентификатор склада получателя [String]
   "isOpenCarSender": false,#Растентовка отправителя [Boolean]
   "isOpenCarReceiver": false,# Растентовка получателя [Boolean]
   "isHyperMarket": false,# признак гипермаркета [Boolean]
   "plannedDateTime": "2023-02-28T14:00:00",# ОБЯЗАТЕЛЬНО!!! Дата и время планируемой передачи груза в ПЭК (по часовому поясу филиала отправления) [DateTime]: при заказанном заборе - дата и время начала интервала забора груза;
                                            # (не позднее 4-х часов до окончания работы филиала),
                                            # при самопривозе - дата и время плановой сдачи груза на склад ПЭК
   "isInsurance": true,# Страхование [Boolean]
   "isInsurancePrice": 234.15,# Стоимость груза (сумма, на которую будет застрахован груз), руб [Number]
   "isPickUp": false,# Нужен забор [Boolean]
   "isDelivery": false,# Нужна доставка [Boolean]
   "needReturnDocuments": false,# Возврат документов [Boolean] (см. комментарии)
   "needArrangeTransportationDocuments": false,# Организация перевозки сопроводительных документов [Boolean] (см. комментарии)
   "senderDistanceType": 0,# В актуальной версии протокола не используется [Number]
   "receiverDistanceType": 0,# В актуальной версии протокола не используется [Number]
   "pickupServices": {# Погрузочно-разгрузочные работы при заборе
       "isLoading": true,# Расчитывать ПРР [Boolean]
       "floor": 0,# Поднять/спустить на этаж [Number]
       "carryingDistance": 0,#  Перенести груз (в метрах) [Number]
       "isElevator": false#  Подъем на лифте [Boolean]
    },
   "deliveryServices": {# Погрузочно-разгрузочные работы при доставке
       "isLoading": true,# Расчитывать ПРР [Boolean]
       "floor": 3,# Поднять/спустить на этаж [Number]
       "carryingDistance": 50,# Перенести груз (в метрах) [Number]
       "isElevator": true# Подъем на лифте [Boolean]
   },
   "pickup": {
        "address" : "Россия, Москва, улица Большая Лубянка, 2",# Обязательный параметр для расчёта забора груза. Для успешного определения
                                                               # координат адреса забора, передавайте все составляющие адреса в такой
                                                               # последовательности: страна, регион, район, населенный пункт, улица, дом,
                                                               # квартира и разделяйте их запятыми. Запрещено в данной строке указывать
                                                               # данные, не являющиеся частью адреса (см. комментарии)
        "coordinates" : {# Необязательный параметр координат адреса (используется для точности вычислений)
            "latitude" : "55.761011",
            "longitude" : "37.627553"
        }
   },
   "delivery" : {
        "address" : "Россия, Москва, улица Большая Лубянка, 2",# Обязательный параметр для расчёта доставки груза. Для успешного определения
                                                               # координат адреса доставки, передавайте все составляющие адреса в такой
                                                               # последовательности: страна, регион, район, населенный пункт, улица, дом,
                                                               # квартира и разделяйте их запятыми. Запрещено в данной строке указывать
                                                               # данные, не являющиеся частью адреса (см. комментарии)
        "coordinates" : {# Необязательный параметр координат адреса (используется для точности вычислений)
            "latitude" : "55.761011",
            "longitude" : "37.627553"
        }
   },
   "counterpart": {# Данные контрагента для расчета с учетом спецусловий
        "inn": "1234567890",# ИНН контрагента [String]
        "kpp": "123456789",# КПП контрагента [String], поле необязательно
        "counterpartClientCard": "1234567890",# Номер карты клиента [String]
        "whoMakesCalculation": [1,2,3,4]# Роль заказчика перевозки  [Array][Number]. (Кто производит расчет:
                                        # 1 - отправитель, 2 - получатель, 3 - плательщик, 4 - не участник перевозки)
   },
   "cargos": [{# Данные о грузе/грузоместах груза (см. комментарии) [Array]
               # Для корректного расчёта передавайте не более двух знаков после запятой,
               # На нашей стороне округление выполняется до сотых в большую сторону.
               # Например, 0.00041 будет 0.01;  1.871001 будет 1.88
      "length": 3.2,# Длина груза, м [Number]. Необязательно, если передан maxSize
      "width": 0.9,# Ширина груза, м [Number]. Необязательно, если передан maxSize
      "height": 1.5,# Высота груза, м [Number]. Необязательно, если передан maxSize
      "volume": 4.32,# Объем груза, м3 [Number]. Необязательно, если переданы длина, ширина, высота.
      "maxSize": 3.2,# Максимальный габарит, м [Number]. Необязательно, если переданы длина, ширина, высота (которые имеют приоритет).
      "isHP": false,# Защитная транспортировочная упаковка [Boolean]. Необязательно
      "sealingPositionsCount": 0,# Количество мест для пломбировки - пломб (применяются для небольших мест или сложных грузов)[Number]. Необязательно.
      "weight": 622.5# Вес, кг [Number]. Обязательно.
      "maPlaceWeight": 50.2 [Number]# максимальный вес грузоместа. Необязательно.
                                     # Рекомендуем передавать только в массиве с одним элементом
                                     # для общих параметров всего груза.
   }]
}

## Метод доступных способов перевозки

/typesOfDelivery/ — продукты и тарифы перевозки
/all/ — перечень актуальных продуктов/тарифов перевозки, которые поддерживаются в API



In [ ]:
import requests
from requests.auth import HTTPBasicAuth
import json

# --- Настройки запроса ---
USERNAME = "..."  # Логин от ЛК ПЭК
API_KEY = "..."            # Ключ из ЛК ПЭК
URL = "https://kabinet.pecom.ru/api/v1/typesOfDelivery/all/"

# --- Отправка запроса ---
headers = {
    "Content-Type": "application/json;charset=utf-8"
}

print("Отправка запроса к API ПЭК...")
try:
    response = requests.post(
        URL,
        auth=HTTPBasicAuth(USERNAME, API_KEY),
        headers=headers,
        json={}
    )

    # Проверка HTTP-статуса
    response.raise_for_status()
    print("\nЗапрос успешно выполнен. Полный ответ API:\n")

    # Форматировние JSON-ответа
    parsed = response.json()
    print(json.dumps(parsed, ensure_ascii=False, indent=2))

except requests.exceptions.HTTPError as errh:
    print(f"\nHTTP ошибка: {errh}")
    print(f"Статус код: {response.status_code}")
    print("Ответ сервера:")
    print(response.text)

except Exception as e:
    print(f"\nПроизошла ошибка: {e}")
    if 'response' in locals():
        print("Ответ сервера:")
        print(response.text)

## Выгрузка справочника филиалов по форме ПЭКа: все филиалы, ПВЗ, с графиками работы и схемой проезда

In [ ]:
#выгрузка справочника филиалов по форме ПЭКа: все филиалы, ПВЗ, с графиками работы и схемой проезда
import requests
from requests.auth import HTTPBasicAuth
import pandas as pd
import json

# --- Настройки запроса ---
USERNAME = "..."# Логин от ЛК ПЭК
API_KEY = "..."# Ключ из ЛК ПЭК
URL = "https://kabinet.pecom.ru/api/v1/branches/all/"

# --- Отправка запроса ---
headers = {"Content-Type": "application/json;charset=utf-8"}
response = requests.post(
    URL,
    auth=HTTPBasicAuth(USERNAME, API_KEY),
    headers=headers,
    json={}
)

# --- Обработка ответа ---
if response.status_code == 200:
    data = response.json()

    # Преобразую JSON в таблицу (DataFrame)
    df = pd.DataFrame(data['branches'])

    # Сохранение в Excel
    excel_file = "pecom_branches.xlsx"
    df.to_excel(excel_file, index=False, engine='openpyxl')

    print(f"Данные успешно сохранены в файл: {excel_file}")
    print(f"Кол-во записей: {len(df)}")

    # Вывод первых строк таблицы
    print("\nПервые 5 строк данных:")
    print(df.head())

else:
    print(f"Ошибка {response.status_code}: {response.text}")

##Выгрузка ID основных терминалов для формирования справочников

In [ ]:
#Выгрузка ID основных терминалов для формирования справочников

import requests
from requests.auth import HTTPBasicAuth
import pandas as pd

# Данные ЛК для доступа через API
USERNAME = "..."
API_KEY = "..."
url = "https://kabinet.pecom.ru/api/v1/branches/all/"
headers = {"Content-Type": "application/json;charset=utf-8"}

# запрос к API
response = requests.post(
    url,
    auth=HTTPBasicAuth(USERNAME, API_KEY),
    headers=headers,
    json={}
)

if response.status_code == 200:
    data = response.json()
    results = []

# функция выбора нужных данных (основные терминалы, названия, и их id) и формировние датафрейма
    for branch in data["branches"]:
        for division in branch.get("divisions", []):
            if division.get("departmentType") == "Основное отделение филиала компании":
                warehouses = division.get("warehouses", [])
                if warehouses:
                    warehouse = warehouses[0]
                    results.append({
                        "Id_терминала": division.get("cityId", ""),
                        "название_терминала": division.get("name", ""),
                        "deliveryAddress": warehouse.get("address", "")
                    })

    df = pd.DataFrame(results)

    # Вывод информации в консоль
    print("DataFrame создан успешно!")
    print(f"Количество записей: {len(df)}")
    print("\nПервые 5 строк:")
    print(df.head())

    # Сохранение в Excel
    excel_filename = "cities.xlsx"
    df.to_excel(excel_filename, index=False, engine='openpyxl')
    print(f"\nДанные успешно сохранены в файл: {excel_filename}")

else:
    print(f"Ошибка {response.status_code}: {response.text}")

## С автозапуском с момента окончания сбора для всех направлений

In [ ]:
# -*- coding: utf-8 -*-
# Минимальная надёжная версия: чекпойнты в Google Drive, возобновление с последней комбинации,
# дата plannedDateTime = ближайший понедельник не ранее +6 дней (12:00), формат Excel по форме мониторинга.

import os
import json
import time
import shutil
import requests
import pandas as pd
from datetime import datetime, timedelta
from requests.auth import HTTPBasicAuth

# ===================== Пользовательские настройки =====================
USERNAME = "..."
API_KEY = "..."
URL = "https://kabinet.pecom.ru/api/v1/calculator/calculateprice/"

EXCEL_FILE = "cities_to.xlsx"           # вход: города отправления
CITIES_FROM_FILE = "cities_from.xlsx"   # вход: города получения

# Папка в Google Drive для чекпойнтов и итогов
DRIVE_DIR = "/content/drive/MyDrive/pecom_exports"

# Имена файлов в Drive
DRIVE_PROGRESS_FILE = os.path.join(DRIVE_DIR, "mt_progress.json")
DRIVE_CACHE_FILE    = os.path.join(DRIVE_DIR, "mt_results_cache.json")
DRIVE_RESULTS_XLSX  = os.path.join(DRIVE_DIR, "mt_results.xlsx")

# Ограничение скорости и retry
REQUESTS_PER_MINUTE = 100
MIN_INTERVAL = 60 / REQUESTS_PER_MINUTE  # 0.6 сек
MAX_RETRIES = 5
RETRY_DELAY = 5
RETRY_BACKOFF = 2

# Сколько запросов между промежуточной записью Excel в Drive (кроме пер-записи cache/progress, которые идут каждый запрос)
SAVE_EVERY_N_REQUESTS = 1000

# ===================== Вспомогательные функции =====================

def mount_drive():
    from google.colab import drive  # доступно в Colab
    drive.mount('/content/drive', force_remount=False)
    os.makedirs(DRIVE_DIR, exist_ok=True)

def compute_planned_datetime(days_ahead=6, hh=12, mm=0):
    base = datetime.now() + timedelta(days=days_ahead)
    shift = (0 - base.weekday()) % 7  # Monday=0
    planned = base + timedelta(days=shift)
    planned = planned.replace(hour=hh, minute=mm, second=0, microsecond=0)
    return planned.isoformat(timespec="seconds")

BASE_PARAMS_TEMPLATE = {
    "types": [3],
    "isInsurance": True,
    "isPickUp": True,
    "isDelivery": True
}

CARGO_VARIANTS = [
    {"weight": 1, "volume": 0.02, "label": "1кг"},
    {"weight": 5, "volume": 0.02, "label": "5кг"},
    {"weight": 15, "volume": 0.02, "label": "15кг"},
    {"weight": 35, "volume": 0.02, "label": "35кг"},
    {"weight": 36, "volume": 0.02, "label": "мин"},
    {"weight": 100, "volume": 0.1, "label": "100кг"},
    {"weight": 150, "volume": 0.1, "label": "150кг"},
    {"weight": 250, "volume": 0.5, "label": "250кг"},
    {"weight": 500, "volume": 0.5, "label": "500кг"},
    {"weight": 750, "volume": 1.0, "label": "750кг"},
    {"cargos": [{"weight": 500, "volume": 0.01}, {"weight": 500, "volume": 0.01}], "label": "1000кг"},
    {"cargos": [{"weight": 500, "volume": 0.01}, {"weight": 500, "volume": 0.01}, {"weight": 500, "volume": 0.01}], "label": "1500кг"},
    {"cargos": [{"weight": 500, "volume": 1.0}, {"weight": 500, "volume": 1.0}, {"weight": 500, "volume": 1.0}, {"weight": 500, "volume": 1.0}], "label": "2000кг"},
    {"cargos": [{"weight": 500, "volume": 1.0}, {"weight": 500, "volume": 1.5}, {"weight": 500, "volume": 1.0}, {"weight": 500, "volume": 1.0}, {"weight": 500, "volume": 1.0}, {"weight": 500, "volume": 1.0}], "label": "3000кг"},
    {"cargos": [{"weight": 500, "volume": 1.0}, {"weight": 500, "volume": 1.5}, {"weight": 500, "volume": 1.0}, {"weight": 500, "volume": 1.0}, {"weight": 500, "volume": 1.0}, {"weight": 501, "volume": 1.0}], "label": "3001кг"},
    {"weight": 1, "volume": 0.14, "label": "0,14м3"},
    {"weight": 5, "volume": 0.4, "label": "0,4м3"},
    {"weight": 15, "volume": 0.6, "label": "0,6м3"},
    {"weight": 35, "volume": 1, "label": "1м3"},
    {"cargos": [{"weight": 50, "volume": 1.0}, {"weight": 50, "volume": 1}], "label": "2м3"},
    {"cargos": [{"weight": 50, "volume": 1.0}, {"weight": 50, "volume": 1}, {"weight": 50, "volume": 1.0}], "label": "3м3"},
    {"cargos": [{"weight": 50, "volume": 1.0}, {"weight": 50, "volume": 1.0}, {"weight": 50, "volume": 1.0}, {"weight": 50, "volume": 1.0}], "label": "4м3"},
    {"cargos": [{"weight": 50, "volume": 1.0}, {"weight": 50, "volume": 1.0}, {"weight": 50, "volume": 1.0}, {"weight": 50, "volume": 1.0}, {"weight": 50, "volume": 1.0}, {"weight": 50, "volume": 1.0}], "label": "6м3"},
    {"cargos": [{"weight": 50, "volume": 1.5}, {"weight": 50, "volume": 1.5}, {"weight": 50, "volume": 1.5}, {"weight": 50, "volume": 1.5}, {"weight": 50, "volume": 1.5}, {"weight": 10, "volume": 0.5}], "label": "8м3"},
    {"cargos": [{"weight": 50, "volume": 1.0}, {"weight": 50, "volume": 1.0}, {"weight": 50, "volume": 1.0}, {"weight": 50, "volume": 1.5}, {"weight": 50, "volume": 1.5}, {"weight": 50, "volume": 1.5}, {"weight": 50, "volume": 1.5}, {"weight": 50, "volume": 1.5}, {"weight": 50, "volume": 1.5}], "label": "12м3"},
    {"cargos": [{"weight": 50, "volume": 1.0}, {"weight": 50, "volume": 1.0}, {"weight": 50, "volume": 1.0}, {"weight": 50, "volume": 1.5}, {"weight": 50, "volume": 1.5}, {"weight": 50, "volume": 1.5}, {"weight": 50, "volume": 1.5}, {"weight": 50, "volume": 1.5}, {"weight": 50, "volume": 1.6}], "label": "12,1м3"}
]

INSURANCE_PRICES = [100]

def load_cities_to():
    df = pd.read_excel(EXCEL_FILE)
    return df[['receiverWarehouseId', 'deliveryAddress', 'receiverCity']].to_dict('records')

def load_cities_from():
    df = pd.read_excel(CITIES_FROM_FILE)
    return df[['senderCity', 'senderWarehouseId', 'pickupAddress']].to_dict('records')

def find_autotransport_cost(services):
    if services is None:
        return None
    for service in services:
        if service.get('info') == 'Автоперевозка':
            return service.get('cost')
    return None

def make_request_with_retry(url, auth, headers, json_data, max_retries=MAX_RETRIES, initial_delay=RETRY_DELAY):
    for attempt in range(max_retries + 1):
        try:
            response = requests.post(url, auth=auth, headers=headers, json=json_data, timeout=30)
            return response
        except (requests.exceptions.ConnectTimeout,
                requests.exceptions.ConnectionError,
                requests.exceptions.ReadTimeout,
                requests.exceptions.Timeout) as e:
            if attempt < max_retries:
                wait_time = initial_delay * (RETRY_BACKOFF ** attempt)
                print(f"Ошибка соединения (попытка {attempt + 1}/{max_retries}): {e}")
                print(f"Повтор через {wait_time} сек...")
                time.sleep(wait_time)
            else:
                print(f"Все попытки завершились ошибкой: {e}")
                raise e
    return None

def safe_divide(value, divisor):
    try:
        if isinstance(value, (int, float)) and value is not None and float(divisor) != 0:
            return value / divisor
        return value
    except Exception:
        return value

# ---------- Чтение/запись прогресса и кэша в Google Drive ----------
def load_progress():
    if os.path.exists(DRIVE_PROGRESS_FILE):
        try:
            with open(DRIVE_PROGRESS_FILE, "r", encoding="utf-8") as f:
                return json.load(f)
        except Exception as e:
            print(f"Не удалось прочитать прогресс: {e}")
    return {"processed": 0}

def save_progress(progress):
    tmp = DRIVE_PROGRESS_FILE + ".tmp"
    with open(tmp, "w", encoding="utf-8") as f:
        json.dump(progress, f, ensure_ascii=False, indent=2)
        f.flush()
        os.fsync(f.fileno())
    os.replace(tmp, DRIVE_PROGRESS_FILE)

def load_cache():
    # cache = {"rows": [row_data,...], "partials": {key: {"senderCity":..,"receiverCity":..,"costs":{label:value}}}}
    if os.path.exists(DRIVE_CACHE_FILE):
        try:
            with open(DRIVE_CACHE_FILE, "r", encoding="utf-8") as f:
                return json.load(f)
        except Exception as e:
            print(f"Не удалось прочитать кэш: {e}")
    return {"rows": [], "partials": {}}

def save_cache(cache):
    tmp = DRIVE_CACHE_FILE + ".tmp"
    with open(tmp, "w", encoding="utf-8") as f:
        json.dump(cache, f, ensure_ascii=False, indent=2)
        f.flush()
        os.fsync(f.fileno())
    os.replace(tmp, DRIVE_CACHE_FILE)

def partial_key(sender_id, receiver_id, insurance):
    return f"{sender_id}|{receiver_id}|{insurance}"

# ---------- Формирование Excel строго как в исходнике ----------
def save_results_to_excel_like_original(rows, excel_path):
    if not rows:
        return
    df = pd.DataFrame(rows)

    base_columns = ["Город отправления", "Город получателя"]
    weight_columns = [
        "1кг", "5кг", "15кг", "35кг", "мин",
        "100кг", "150кг", "250кг", "500кг", "750кг",
        "1000кг", "1500кг", "2000кг", "3000кг", "3001кг"
    ]
    volume_columns = [
        "0,004 м3", "0,02 м3", "0,06 м3",
        "0,14м3", "0,4м3", "0,6м3", "1м3",
        "2м3", "3м3", "4м3", "6м3", "8м3", "12м3", "12,1м3"
    ]

    ordered_columns = base_columns + weight_columns + volume_columns
    existing_columns = [col for col in ordered_columns if col in df.columns]
    missing_columns = [col for col in df.columns if col not in existing_columns]
    final_columns = existing_columns + missing_columns
    df = df[final_columns]

    # вычисляемые столбцы (как в исходнике)
    if "100кг" in df.columns: df["до 100 кг"] = df["100кг"].apply(lambda x: safe_divide(x, 100))
    if "150кг" in df.columns: df["до 150 кг"] = df["150кг"].apply(lambda x: safe_divide(x, 150))
    if "250кг" in df.columns: df["до 250 кг"] = df["250кг"].apply(lambda x: safe_divide(x, 250))
    if "500кг" in df.columns: df["до 500 кг"] = df["500кг"].apply(lambda x: safe_divide(x, 500))
    if "750кг" in df.columns: df["до 750 кг"] = df["750кг"].apply(lambda x: safe_divide(x, 750))
    if "1000кг" in df.columns: df["до 1000 кг"] = df["1000кг"].apply(lambda x: safe_divide(x, 1000))
    if "1500кг" in df.columns: df["до 1500 кг"] = df["1500кг"].apply(lambda x: safe_divide(x, 1500))
    if "2000кг" in df.columns: df["до 2000 кг"] = df["2000кг"].apply(lambda x: safe_divide(x, 2000))
    if "3000кг" in df.columns: df["до 3000 кг"] = df["3000кг"].apply(lambda x: safe_divide(x, 3000))
    if "3001кг" in df.columns: df["от 3000 кг"] = df["3001кг"].apply(lambda x: safe_divide(x, 3001))

    if "0,4м3" in df.columns: df["до 0,4 м3"] = df["0,4м3"].apply(lambda x: safe_divide(x, 0.4))
    if "0,6м3" in df.columns: df["до 0,6 м3"] = df["0,6м3"].apply(lambda x: safe_divide(x, 0.6))
    if "2м3" in df.columns: df["до 2 м3"] = df["2м3"].apply(lambda x: safe_divide(x, 2))
    if "3м3" in df.columns: df["до 3 м3"] = df["3м3"].apply(lambda x: safe_divide(x, 3))
    if "4м3" in df.columns: df["до 4 м3"] = df["4м3"].apply(lambda x: safe_divide(x, 4))
    if "6м3" in df.columns: df["до 6 м3"] = df["6м3"].apply(lambda x: safe_divide(x, 6))
    if "8м3" in df.columns: df["до 8 м3"] = df["8м3"].apply(lambda x: safe_divide(x, 8))
    if "12м3" in df.columns: df["до 12 м3"] = df["12м3"].apply(lambda x: safe_divide(x, 12))
    if "12,1м3" in df.columns: df["от 12 м3"] = df["12,1м3"].apply(lambda x: safe_divide(x, 12.1))

    simple_rename = {
        "1кг": "до 1 кг",
        "5кг": "до 5 кг",
        "15кг": "до 15 кг",
        "35кг": "до 35 кг",
        "мин": "мин_кг",
        "0,004 м3": "до 0,004 м3",
        "0,02 м3": "до 0,02 м3",
        "0,06 м3": "до 0,06 м3",
        "0,14м3": "до 0,14 м3",
        "1м3": "до 1 м3"
    }
    df.rename(columns=simple_rename, inplace=True)

    final_ordered_columns = [
        "Город отправления", "Город получателя", "Объявленная стоимость",
        "до 1 кг", "до 5 кг", "до 15 кг", "до 35 кг", "мин_кг",
        "до 100 кг", "до 150 кг", "до 250 кг", "до 500 кг", "до 750 кг",
        "до 1000 кг", "до 1500 кг", "до 2000 кг", "до 3000 кг", "от 3000 кг",
        "до 0,004 м3", "до 0,02 м3", "до 0,06 м3",
        "до 0,14 м3", "до 0,4 м3", "до 0,6 м3", "до 1 м3",
        "до 2 м3", "до 3 м3", "до 4 м3", "до 6 м3", "до 8 м3", "до 12 м3", "от 12 м3"
    ]
    existing_final = [c for c in final_ordered_columns if c in df.columns]
    df = df[existing_final]

    # Пишем сразу в Google Drive
    df.to_excel(DRIVE_RESULTS_XLSX, sheet_name="Автоперевозка", index=False)

# ---------- Основная логика ----------
def calculate_for_cities(cities_to):
    mount_drive()

    if not os.path.exists(EXCEL_FILE):
        print(f"Ошибка: Файл {EXCEL_FILE} не найден!")
        return
    if not os.path.exists(CITIES_FROM_FILE):
        print(f"Ошибка: Файл {CITIES_FROM_FILE} не найден!")
        print("Файл должен содержать колонки: senderCity, senderWarehouseId, pickupAddress")
        return

    cities_from = load_cities_from()
    plannedDateTime = compute_planned_datetime()

    progress = load_progress()                # {"processed": int}
    cache = load_cache()                      # {"rows":[...], "partials":{key:{...}}}

    total_combinations = len(cities_from) * len(cities_to) * len(CARGO_VARIANTS) * len(INSURANCE_PRICES)
    print(f"Всего возможных комбинаций: {total_combinations}")

    processed_target = int(progress.get("processed", 0))
    global_counter = 0
    request_count = 0
    failed_requests = 0

    for city_from in cities_from:
        sender_city_name = city_from["senderCity"]
        sender_warehouse_id = city_from["senderWarehouseId"]
        pickup_address = city_from["pickupAddress"]

        base_params = {
            **BASE_PARAMS_TEMPLATE,
            "senderWarehouseId": sender_warehouse_id,
            "pickup": {"address": pickup_address},
            "plannedDateTime": plannedDateTime
        }

        for city_to_item in cities_to:
            receiver_city_name = city_to_item.get("receiverCity", "")
            receiver_address = city_to_item["deliveryAddress"]
            receiver_id = city_to_item["receiverWarehouseId"]

            if sender_city_name.strip().lower() == receiver_city_name.strip().lower():
                continue

            for insurance_price in INSURANCE_PRICES:
                key = partial_key(sender_warehouse_id, receiver_id, insurance_price)
                # Восстанавливаем частичный прогресс по маршруту
                if key not in cache["partials"]:
                    cache["partials"][key] = {
                        "senderCity": sender_city_name,
                        "receiverCity": receiver_city_name,
                        "costs": {}
                    }

                for cargo_variant in CARGO_VARIANTS:
                    cargo_label = cargo_variant["label"]

                    # Пропускаем уже обработанные комбинации
                    if global_counter < processed_target:
                        global_counter += 1
                        continue

                    request_count += 1
                    global_counter += 1
                    start_time = time.time()

                    # Сбор грузомест
                    if "cargos" in cargo_variant:
                        cargos_list = cargo_variant["cargos"]
                    else:
                        cargos_list = [{"weight": cargo_variant["weight"], "volume": cargo_variant["volume"]}]

                    # Запрос
                    request_data = {
                        **base_params,
                        "cargos": cargos_list,
                        "isInsurancePrice": insurance_price,
                        "receiverWarehouseId": receiver_id,
                        "delivery": {"address": receiver_address}
                    }

                    print(f"\nЗапрос {request_count}: {sender_city_name} -> {receiver_city_name} ({cargo_label}) | {plannedDateTime}")

                    try:
                        response = make_request_with_retry(
                            URL,
                            auth=HTTPBasicAuth(USERNAME, API_KEY),
                            headers={"Content-Type": "application/json;charset=utf-8"},
                            json_data=request_data
                        )
                        print(f"HTTP статус: {response.status_code if response else 'None'}")

                        json_response = {}
                        error_msg = None
                        autotransport_cost = None

                        if response is not None and response.content:
                            try:
                                json_response = response.json()
                            except json.JSONDecodeError:
                                error_msg = "JSON decode error"
                        else:
                            error_msg = "Empty response"

                        if response is not None and not error_msg:
                            try:
                                response.raise_for_status()
                            except requests.exceptions.HTTPError as e:
                                error_msg = f"HTTP Error: {str(e)}"

                        if not error_msg:
                            if not json_response.get('hasError', True):
                                for transfer in json_response.get('transfers', []):
                                    services = transfer.get('services', [])
                                    cost = find_autotransport_cost(services)
                                    if cost is not None:
                                        autotransport_cost = cost
                                        break
                            else:
                                error_msg = json_response.get('errorMessage', 'Unknown error')

                        # Логика записи результата для одной комбинации
                        if error_msg:
                            cache["partials"][key]["costs"][cargo_label] = error_msg
                            failed_requests += 1
                            print(f"Автоперевозка: ошибка -> {error_msg}")
                        else:
                            cache["partials"][key]["costs"][cargo_label] = autotransport_cost
                            print(f"Автоперевозка (в файл): {autotransport_cost}")

                    except (requests.exceptions.ConnectTimeout,
                            requests.exceptions.ConnectionError,
                            requests.exceptions.ReadTimeout,
                            requests.exceptions.Timeout) as e:
                        msg = f'Connection failed after {MAX_RETRIES} retries: {str(e)}'
                        cache["partials"][key]["costs"][cargo_label] = msg
                        failed_requests += 1
                        print(f"Автоперевозка: ошибка -> {msg}")

                    except Exception as e:
                        msg = f'Unexpected error: {str(e)}'
                        cache["partials"][key]["costs"][cargo_label] = msg
                        failed_requests += 1
                        print(f"Автоперевозка: ошибка -> {msg}")

                    # 1) Обновляем счётчик прогресса ПОСЛЕ каждого запроса
                    progress["processed"] = global_counter
                    save_progress(progress)

                    # 2) Сохраняем кэш частичных результатов ПОСЛЕ каждого запроса
                    save_cache(cache)

                    # Ограничение скорости
                    elapsed = time.time() - start_time
                    if elapsed < MIN_INTERVAL:
                        time.sleep(MIN_INTERVAL - elapsed)

                    # Промежуточный Excel в Drive по таймеру
                    if (request_count % SAVE_EVERY_N_REQUESTS) == 0 and cache["rows"]:
                        save_results_to_excel_like_original(cache["rows"], DRIVE_RESULTS_XLSX)

                # Когда все грузы по маршруту обработаны — формируем итоговую строку
                costs = cache["partials"][key]["costs"]
                # Если чего-то не хватило, оставим "N/A" — но при возобновлении недостающее будет дозаполнено
                row_data = {
                    "Город отправления": cache["partials"][key]["senderCity"],
                    "Город получателя": cache["partials"][key]["receiverCity"]
                }
                for cargo_variant in CARGO_VARIANTS:
                    lbl = cargo_variant["label"]
                    row_data[lbl] = costs.get(lbl, "N/A")
                # объёмные минималки на основе весовых
                row_data["0,004 м3"] = costs.get("1кг", "N/A")
                row_data["0,02 м3"] = costs.get("1кг", "N/A")
                row_data["0,06 м3"] = costs.get("15кг", "N/A")

                # Перенос в завершённые строки
                cache["rows"].append(row_data)
                del cache["partials"][key]
                save_cache(cache)

    # Итоговая статистика
    success_rate = ((request_count - failed_requests) / request_count * 100) if request_count > 0 else 0
    print(f"\nСтатистика выполнения:\n  Всего запросов: {request_count}\n  Успешных: {request_count - failed_requests}\n  Неудачных: {failed_requests}\n  Процент успеха: {success_rate:.1f}%")

    # Финальный Excel
    if cache["rows"]:
        save_results_to_excel_like_original(cache["rows"], DRIVE_RESULTS_XLSX)
        print(f"Итог сохранён в: {DRIVE_RESULTS_XLSX}")
    else:
        print("Нет данных для сохранения")

# ===================== Запуск =====================
if __name__ == "__main__":
    if not os.path.exists(EXCEL_FILE):
        print(f"Ошибка: Файл {EXCEL_FILE} не найден! Загрузите cities_to.xlsx в текущую директорию.")
    elif not os.path.exists(CITIES_FROM_FILE):
        print(f"Ошибка: Файл {CITIES_FROM_FILE} не найден! Загрузите cities_from.xlsx (senderCity, senderWarehouseId, pickupAddress).")
    else:
        cities_to = load_cities_to()
        calculate_for_cities(cities_to)
